<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Analyse, Conception et Implémentation d'Algorithmes</span>

---

# Le Sudoku comme CSP

<span style="color:gray; font-style:italic;">Les notebooks de cette section présentent différentes approches pour résoudre des puzzles Sudoku.</span>

# Sudoku-7-Norvig-Python : Solveur de Peter Norvig (Python)

**Navigation** : [<< AIMA CSP](Sudoku-6-AIMA-Python.ipynb) | [Index](../Sudoku-Index.ipynb) | [Python Modeling >>](Sudoku-8-ORTools-Python.ipynb)

---

## Objectifs

- Comprendre l'approche élégante de Peter Norvig pour résoudre le Sudoku
- Implémenter un solveur basé sur la propagation de contraintes et le backtracking
- Comparer les performances sur différents niveaux de difficulté

## Prérequis

- Python 3.10+
- Concepts de base en programmation par contraintes

## Durée estimée

15 minutes

## Configuration et Imports

In [1]:
# Imports
import time
from typing import Dict, Set, Tuple, Optional, List

print('Solveur Sudoku - Approche Norvig')

Solveur Sudoku - Approche Norvig


## Introduction : L'approche Norvig

Peter Norvig a proposé en 2006 une solution élégante en ~80 lignes de Python. L'idée clé :

1. **Représentation** : Chaque case contient l'ensemble des valeurs possibles
2. **Propagation** : Éliminer les valeurs impossibles (Constraint Propagation)
3. **Backtracking** : Si la propagation bloque, choisir une case et essayer une valeur

### Heuristiques clés

- **MRV (Minimum Remaining Values)** : Choisir la case avec le moins de valeurs possibles
- **Forward Checking** : Propager immédiatement les conséquences de chaque choix

> **Lien** : [Solving Every Sudoku Puzzle](https://norvig.com/sudoku.html) par Peter Norvig

## Définition des constantes

Nous définissons la structure du Sudoku : 81 cases, 27 unités (9 lignes, 9 colonnes, 9 blocs).

In [2]:
# Définition des constantes
digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits

squares  = [r + c for r in rows for c in cols]

# Unites : lignes, colonnes, blocs
unitlist = (
    [['A'+c, 'B'+c, 'C'+c, 'D'+c, 'E'+c, 'F'+c, 'G'+c, 'H'+c, 'I'+c] for c in cols] +
    [[r+'1', r+'2', r+'3', r+'4', r+'5', r+'6', r+'7', r+'8', r+'9'] for r in rows] +
    [[r+c for r in rs for c in cs] for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
)

# Dictionnaire : square -> liste des 3 unités
units = {s: [u for u in unitlist if s in u] for s in squares}

# Dictionnaire : square -> ensemble des 20 pairs
peers = {s: set(sum(units[s], [])) - {s} for s in squares}

print(f'Carres: {len(squares)}')
print(f'Unites: {len(unitlist)}')
print(f'Pairs de A1: {len(peers["A1"])}')
print(f'Unites de A1: {units["A1"]}')

Carres: 81
Unites: 27
Pairs de A1: 20
Unites de A1: [['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'], ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]


## Propagation des contraintes

Les deux opérations fondamentales :

1. **assign(values, square, digit)** : Assigne un chiffre à une case
   - Élimine ce chiffre de tous les pairs de la case

2. **eliminate(values, square, digit)** : Élimine un chiffre possible d'une case
   - Si une case n'a plus qu'une valeur, l'éliminer des pairs
   - Si une unité n'a qu'une place pour un chiffre, l'assigner

### Règles de propagation

| Règle | Description |
|-------|-------------|
| **Peer Elimination** | Si A1 = 1, alors 1 est éliminé de tous les pairs de A1 |
| **Singleton Unit** | Si seule la case A1 peut contenir 1 dans son bloc, alors A1 = 1 |

In [3]:
def assign(values: Dict[str, str], s: str, d: str) -> Optional[Dict[str, str]]:
    """Assigne d à s en éliminant toutes les autres valeurs.
    
    Renvoie values si succès, None si contradiction.
    """
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return None

def eliminate(values: Dict[str, str], s: str, d: str) -> bool:
    """Élimine d de values[s]; propage les conséquences."""
    if d not in values[s]:
        return True  # Déjà éliminé
    
    values[s] = values[s].replace(d, '')
    
    # (1) Si une case n'a qu'une valeur, l'éliminer des pairs
    if len(values[s]) == 0:
        return False  # Contradiction: plus de valeurs
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    
    # (2) Si une unité n'a qu'une place pour d, l'y assigner
    for u in units[s]:
        dplaces = [s2 for s2 in u if d in values[s2]]
        if len(dplaces) == 0:
            return False  # Contradiction: pas de place pour d
        elif len(dplaces) == 1:
            if not assign(values, dplaces[0], d):
                return False
    return True

## Parsing d'une grille

Une grille est représentée comme un dictionnaire `{square: digits}` où `digits` est :
- Un chiffre ('1'-'9') si la case est résolue
- Une chaîne de chiffres possibles si la case n'est pas résolue

Format d'entrée : chaîne de 81 caractères (ou avec `.` et `0` pour les cases vides).

In [4]:
def parse_grid(grid: str) -> Optional[Dict[str, str]]:
    """Convertit une grille string en dictionnaire {square: digits}."""
    values = {s: digits for s in squares}  # Toutes les valeurs possibles
    for s, d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return None  # Contradiction
    return values

def grid_values(grid: str) -> Dict[str, str]:
    """Convertit une grille en dictionnaire {square: char}."""
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

# Test
test_grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
print(f'Grille test: {len(test_grid)} caracteres')

Grille test: 81 caracteres


## Résolution avec Backtracking

Si la propagation ne suffit pas, on utilise le backtracking avec l'heuristique MRV :

1. Choisir la case avec le **moins de valeurs possibles** (Minimum Remaining Values)
2. Essayer chaque valeur possible
3. Récursivement résoudre
4. Si échec, backtracker

### Pourquoi MRV ?

- Maximise les chances d'échec rapide (pruning)
- Réduit la profondeur de l'arbre de recherche
- Est souvent couplé avec LCV (Least Constraining Value)

In [5]:
def solve(grid: str) -> Optional[Dict[str, str]]:
    """Résout une grille Sudoku."""
    return search(parse_grid(grid))

def search(values: Optional[Dict[str, str]]) -> Optional[Dict[str, str]]:
    """Backtracking avec heuristique MRV."""
    if values is None:
        return None  # Échec
    if all(len(values[s]) == 1 for s in squares):
        return values  # Résolu !
    
    # Choisir la case avec le moins de valeurs (MRV)
    n, s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    
    # Essayer chaque valeur
    for d in values[s]:
        result = search(assign(values.copy(), s, d))
        if result:
            return result
    return None

## Affichage des résultats

La fonction `display()` formate une grille de manière lisible.

In [6]:
def display(values: Dict[str, str]) -> str:
    """Affiche une grille."""
    width = 1 + max(len(values[s]) for s in squares)
    line = '+'.join(['-' * (width * 3)] * 3)
    result = []
    for r in rows:
        result.append(''.join(values[r + c].center(width) + ('|' if c in '36' else '')
                                for c in cols))
        if r in 'CF':
            result.append(line)
    return '\n'.join(result)

## Test du solveur

Testons le solveur sur une grille facile.

In [7]:
# Test avec une grille facile
easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
print('Puzzle:')
values = parse_grid(easy)
print(display(values))
print('\nResolution...')
start = time.time()
solution = solve(easy)
elapsed = (time.time() - start) * 1000
if solution:
    print(f'\nResolu en {elapsed:.2f} ms')
    print(display(solution))
else:
    print('Pas de solution')

Puzzle:
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 

Resolution...

Resolu en 3.90 ms
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


## Benchmark

Testons le solveur sur des puzzles de différentes difficultés.

In [8]:
from pathlib import Path

PUZZLES_DIR = Path(r'D:\Dev\CoursIA\MyIA.AI.Notebooks\Sudoku\Puzzles')

def load_puzzles(filepath: str, max_puzzles: int = None):
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

def benchmark(puzzles: list, name: str):
    print(f'\nBenchmark: {name} ({len(puzzles)} puzzles)')
    start = time.time()
    solved = 0
    for puzzle in puzzles:
        if solve(puzzle):
            solved += 1
    elapsed = (time.time() - start) * 1000
    print(f'  Resolus: {solved}/{len(puzzles)}')
    print(f'  Temps total: {elapsed:.2f} ms')
    print(f'  Temps moyen: {elapsed/len(puzzles):.2f} ms/puzzle')

# Tests
easy = load_puzzles(str(PUZZLES_DIR / 'Sudoku_Easy51.txt'), max_puzzles=20)
hard = load_puzzles(str(PUZZLES_DIR / 'Sudoku_hardest.txt'))

benchmark(easy, 'Puzzles Faciles')
benchmark(hard, 'Puzzles Difficiles')


Benchmark: Puzzles Faciles (20 puzzles)


  Resolus: 20/20
  Temps total: 63.94 ms
  Temps moyen: 3.20 ms/puzzle

Benchmark: Puzzles Difficiles (11 puzzles)
  Resolus: 11/11
  Temps total: 50.06 ms
  Temps moyen: 4.55 ms/puzzle


## Résumé

### Points clés du solveur Norvig

1. **Propagation** : Réduit les domaines avant la recherche
2. **MRV** : Choix intelligent de la case suivante
3. **Backtracking simple** : Pas de besoin d'algorithmes complexes

### Performances typiques

| Type | Temps moyen | Taux de succès |
|------|-------------|----------------|
| Facile | <1 ms | 100% |
| Moyen | 1-10 ms | 100% |
| Difficile | 10-100 ms | ~100% |

### Références

- [Peter Norvig's Sudoku Solver](https://norvig.com/sudoku.html)
- [Constraint Satisfaction](https://en.wikipedia.org/wiki/Constraint_satisfaction_problem)

---

**Navigation** : [<< AIMA CSP](Sudoku-6-AIMA-Python.ipynb) | [Index](../Sudoku-Index.ipynb) | [OR-Tools >>](Sudoku-8-ORTools-Python.ipynb)